In [1]:
# pip install MetaTrader5

In [2]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import time
import logging


In [3]:
# Set up logging
logging.basicConfig(filename='trading_bot.log', level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')

# Connect to MetaTrader 5
if not mt5.initialize():
    logging.error("initialize() failed")
    mt5.shutdown()

In [4]:

# Login to your account
account_number = 156090030
password = "codewithdark_E0"
server = "Exness-MT5Trial"

authorized = mt5.login(account_number, password, server)
if not authorized:
    print("Failed to connect at account #{}, error code: {}".format(account_number, mt5.last_error()))
else:
    print("Connected to account #{}".format(account_number))

Connected to account #156090030


In [5]:
symbols = ["EURUSDm", "BTCUSDm", "XAUUSDm"]
lot_size = 0.1
max_open_trades_per_symbol = 3
pip_threshold = 10  # Pip threshold for closing trades
take_profit_pips = 20  # Example take profit pips

In [6]:
def get_data(symbol, timeframe, num_bars):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

In [7]:
def calculate_pips_to_price(pips):
    """Converts pips to price"""
    return pips * 0.0001  # Assuming a standard Forex pair, adjust as needed

In [8]:
def place_trade(symbol, action, lot_size, take_profit):
    price = mt5.symbol_info_tick(symbol).ask if action == "buy" else mt5.symbol_info_tick(symbol).bid
    order_type = mt5.ORDER_TYPE_BUY if action == "buy" else mt5.ORDER_TYPE_SELL
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot_size,
        "type": order_type,
        "price": price,
        "deviation": 10,
        "magic": 234000,
        "comment": "Python script order",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC
    }
    result = mt5.order_send(request)
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        logging.error(f"Order send failed for {symbol}, retcode={result.retcode}, comment={result.comment}")
    else:
        logging.info(f"Order sent successfully for {symbol}")


In [9]:
def close_trade(position):
    order_type = mt5.ORDER_TYPE_SELL if position.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
    price = mt5.symbol_info_tick(position.symbol).bid if order_type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(position.symbol).ask
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": position.symbol,
        "volume": position.volume,
        "type": order_type,
        "position": position.ticket,
        "price": price,
        "deviation": 10,
        "magic": 234000,
        "comment": "Close position",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC
    }
    result = mt5.order_send(request)
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        logging.error(f"Close position failed for {position.symbol}, retcode={result.retcode}")
    else:
        logging.info(f"Position closed successfully for {position.symbol}")

In [10]:
def get_open_positions_count(symbol):
    positions = mt5.positions_get(symbol=symbol)
    if positions is None:
        return 0
    return len(positions)

In [11]:
while True:
        for symbol in symbols:
            try:
                # Fetch the last 100 candles
                df = get_data(symbol, mt5.TIMEFRAME_D1, 100)
                if df is None:
                    logging.error("Rates fetch returned None")
                    raise ValueError("Rates fetch returned None")
                
                if len(df) < 2:
                    logging.error(f"Insufficient rates data for {symbol}.")
                    raise ValueError("Insufficient rates data length")

                latest_candle = df.iloc[-2]  # Previous candle
                current_candle = df.iloc[-1]  # Current candle

                current_price = mt5.symbol_info_tick(symbol).last
                action = None
                

                if current_price > current_candle['open']:
                    action = 'buy'
                    take_profit = current_price + calculate_pips_to_price(take_profit_pips)
                elif current_price < current_candle['open']:
                    action = 'sell'
                    take_profit = current_price - calculate_pips_to_price(take_profit_pips)
                
                open_positions_count = get_open_positions_count(symbol)
                logging.info(f"Number of open positions for {symbol}: {open_positions_count}")

                if open_positions_count < max_open_trades_per_symbol:

                    if action:
                        logging.debug(f"Placing trade: {action}")
                        place_trade(symbol, action, lot_size=lot_size, take_profit=take_profit)
                    else:
                        logging.debug("No trade action taken")

                # Close trades based on conditions
                open_positions = mt5.positions_get(symbol=symbol)
                for position in open_positions:
                    # Example condition: Close if profit > $10
                    if position.profit >= 2 :
                        logging.debug(f"Closing trade: {position}")
                        close_trade(position)
            
            except Exception as e:
                logging.error(f"Exception while processing {symbol}: {str(e)}")

        time.sleep(60)  # Wait for 1 minute
    
mt5.shutdown()

KeyboardInterrupt: 

In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import logging
import time

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s: %(levelname)s: %(message)s')

# Initialize the MT5 platform
if not mt5.initialize():
    logging.error("initialize() failed")
    mt5.shutdown()
    exit()

logging.info("Auto trading is enabled in MetaTrader 5 terminal")

# Define the symbols to trade
symbols = ["EURUSDm", "GBPUSDm", "BTCUSDm", "XAUUSDm"]

# Define the lot size and trading parameters
lot_size = 0.1
max_open_trades_per_symbol = 3
take_profit_pips = 20  # Example take profit pips
stop_loss_pips = 10  # Example stop loss pips
trailing_stop_pips = 5  # Example trailing stop loss pips

def get_data(symbol, timeframe, num_bars):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

def calculate_pips_to_price(pips):
    """Converts pips to price"""
    return pips * 0.0001  # Assuming a standard Forex pair, adjust as needed

def place_trade(symbol, action, lot_size):
    price = mt5.symbol_info_tick(symbol).ask if action == "buy" else mt5.symbol_info_tick(symbol).bid
    order_type = mt5.ORDER_TYPE_BUY if action == "buy" else mt5.ORDER_TYPE_SELL
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot_size,
        "type": order_type,
        "price": price,
        "deviation": 10,
        "magic": 234000,
        "comment": "Python script order",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC
    }
    result = mt5.order_send(request)
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        logging.error(f"Order send failed for {symbol}, retcode={result.retcode}, comment={result.comment}")
    else:
        logging.info(f"Order sent successfully for {symbol}")

def close_trade(position):
    order_type = mt5.ORDER_TYPE_SELL if position.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
    price = mt5.symbol_info_tick(position.symbol).bid if order_type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(position.symbol).ask
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": position.symbol,
        "volume": position.volume,
        "type": order_type,
        "position": position.ticket,
        "price": price,
        "deviation": 10,
        "magic": 234000,
        "comment": "Close position"
    }
    result = mt5.order_send(request)
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        logging.error(f"Close position failed for {position.symbol}, retcode={result.retcode}")
    else:
        logging.info(f"Position closed successfully for {position.symbol}")

def rsi_strategy(df):
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['rsi'] = 100 - (100 / (1 + rs))
    if df['rsi'].iloc[-1] < 30:
        return "buy"
    elif df['rsi'].iloc[-1] > 70:
        return "sell"
    return None

def bollinger_bands_strategy(df):
    df['middle_band'] = df['close'].rolling(window=20).mean()
    df['upper_band'] = df['middle_band'] + 2*df['close'].rolling(window=20).std()
    df['lower_band'] = df['middle_band'] - 2*df['close'].rolling(window=20).std()
    if df['close'].iloc[-1] < df['lower_band'].iloc[-1]:
        return "buy"
    elif df['close'].iloc[-1] > df['upper_band'].iloc[-1]:
        return "sell"
    return None

def get_open_positions_count(symbol):
    positions = mt5.positions_get(symbol=symbol)
    if positions is None:
        return 0
    return len(positions)

# Main trading loop
try:
    while True:
        for symbol in symbols:
            try:
                # Fetch the last 100 candles
                df = get_data(symbol, mt5.TIMEFRAME_M1, 100)
                if df is None:
                    logging.error("Rates fetch returned None")
                    raise ValueError("Rates fetch returned None")
                
                if len(df) < 2:
                    logging.error(f"Insufficient rates data for {symbol}.")
                    raise ValueError("Insufficient rates data length")

                latest_candle = df.iloc[-2]  # Previous candle
                current_candle = df.iloc[-1]  # Current candle

                current_price = mt5.symbol_info_tick(symbol).last
                action = None
                take_profit = None
                stop_loss = None

                rsi_signal = rsi_strategy(df)
                bb_signal = bollinger_bands_strategy(df)

                if rsi_signal == 'buy' or bb_signal == 'buy':
                    action = 'buy'
                    take_profit = current_price + calculate_pips_to_price(take_profit_pips)
                    stop_loss = current_price - calculate_pips_to_price(stop_loss_pips)
                elif rsi_signal == 'sell' or bb_signal == 'sell':
                    action = 'sell'
                    take_profit = current_price - calculate_pips_to_price(take_profit_pips)
                    stop_loss = current_price + calculate_pips_to_price(stop_loss_pips)

                open_positions_count = get_open_positions_count(symbol)
                logging.info(f"Number of open positions for {symbol}: {open_positions_count}")

                if open_positions_count < max_open_trades_per_symbol:
                    if action:
                        logging.debug(f"Placing trade: {action}")
                        place_trade(symbol, action, lot_size=lot_size)
                    else:
                        logging.debug("No trade action taken")

                # Close trades based on conditions
                open_positions = mt5.positions_get(symbol=symbol)
                for position in open_positions:
                    # Close if profit >= $2 or loss <= -$0.1
                    if position.profit >= 2 or (position.profit <= -0.1 and position.profit > -1):
                        logging.debug(f"Closing trade: {position}")
                        close_trade(position)
            
            except Exception as e:
                logging.error(f"Exception while processing {symbol}: {str(e)}")

        time.sleep(60)  # Wait for 1 minute

finally:
    mt5.shutdown()
